# Fit Atmosphere Short time scale sequences with Gaussian Processes from Merra2

- author Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date 2025-10-26 :
- last update : 2025-02-27 :  Use mattern in PWV to simulate long-tails
- last update : 2025-02-28 : update FFT and fit with Levy
- last update : 225-03-01 : improve histo fitting : https://stackoverflow.com/questions/35544233/fit-a-curve-to-a-histogram-in-python
- Kernel @usdf **w_2024_50*
- Office emac : mamba_py311
- Home emac : base (conda)
- laptop : conda_py311

**Goal** : Fit the variation of Merra2 parameter impact the transmission

- CO2 fit : https://scikit-learn.org/stable/auto_examples/gaussian_process/plot_gpr_co2.html#sphx-glr-auto-examples-gaussian-process-plot-gpr-co2-py

- Kernels : https://scikit-learn.org/stable/modules/gaussian_process.html#gp-kernels

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import os

In [ ]:
# where are stored the figures
pathfigs = "figsFitGPShortTimeAtmosphereFomMerra2"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
# where are stored the figures
pathdata = "dataFitGPPerAtmosphereFomMerra2"
if not os.path.exists(pathdata):
    os.makedirs(pathdata) 
datatype = ".csv"

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from astropy.coordinates.earth import EarthLocation
from datetime import datetime
from pytz import timezone

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy
from scipy import stats
from scipy.optimize import curve_fit,least_squares


props = dict(boxstyle='round', facecolor='white', alpha=0.5)

In [ ]:
# Remove to run faster the notebook
import ipywidgets as widgets
%matplotlib widget

In [ ]:
from astropy.modeling import models

In [ ]:
from numpy.random import lognormal

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time
from astropy.timeseries import TimeSeries

In [ ]:
# Remove to run faster the notebook
import ipywidgets as widgets
%matplotlib widget

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
np.__version__

In [ ]:
pd.__version__

In [ ]:
from astropy.timeseries import LombScargle

In [ ]:
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import ExpSineSquared
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import WhiteKernel
from sklearn.gaussian_process.kernels import ConstantKernel
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import Kernel, Hyperparameter

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.special import gamma
from scipy.stats import levy_stable,cauchy, laplace, norm

In [ ]:
from astropy.modeling import models, fitting

In [ ]:
class GammaKernel(Kernel):
    """Noyau gamma personnalisé.
        Explications :
       np.clip : Limite les valeurs de distance pour éviter les dépassements lors de l'élévation à la puissance 
       gamma.
       np.nan_to_num : Remplace les valeurs infinies ou NaN par des zéros pour éviter les problèmes 
       lors des calculs ultérieurs.
    Augmentation des itérations : Le nombre d'itérations pour l'optimiseur est augmenté pour améliorer la convergence.
       Ces ajustements devraient aider à réduire les avertissements et à améliorer la stabilité du modèle. Si vous avez d'autres questions ou des problèmes persistants, n'hésitez pas à me le faire savoir !
    """

    def __init__(self, length_scale=1.0, length_scale_bounds=(1e-5, 1e5), gamma=1.0, gamma_bounds=(1e-5, 1e5)):
        self.length_scale = length_scale
        self.length_scale_bounds = length_scale_bounds
        self.gamma = gamma
        self.gamma_bounds = gamma_bounds

    @property
    def hyperparameter_length_scale(self):
        return Hyperparameter("length_scale", "numeric", self.length_scale_bounds)

    @property
    def hyperparameter_gamma(self):
        return Hyperparameter("gamma", "numeric", self.gamma_bounds)

    def __call__(self, X, Y=None, eval_gradient=False):
        if Y is None:
            Y = X
        # Calculer la matrice de distance
        dists = np.abs(X[:, np.newaxis, :] - Y[np.newaxis, :, :])
        # Éviter les dépassements en utilisant np.clip
        dists = np.clip(dists, a_min=1e-10, a_max=10) ** self.gamma
        # Calculer la matrice de covariance
        K = np.exp(-dists.squeeze() / self.length_scale)
        # Remplacer les valeurs infinies ou NaN par 0
        K = np.nan_to_num(K)
        if eval_gradient:
            if not self.hyperparameter_length_scale.fixed:
                K_gradient = (
                    K * dists.squeeze() / self.length_scale**2
                )
                K_gradient = np.nan_to_num(K_gradient)
                return K, np.dstack((K_gradient, K_gradient))
            else:
                return K, np.empty((X.shape[0], X.shape[0], 0))
        else:
            return K

    def diag(self, X):
        return np.ones(X.shape[0])

    def is_stationary(self):
        return True



In [ ]:
class PoissonKernel(Kernel):
    def __init__(self, intensity=1.0):
        self.intensity = intensity

    def __call__(self, X, Y=None, eval_gradient=False):
        X = np.atleast_2d(X)
        Y = X if Y is None else np.atleast_2d(Y)

        dists = np.linalg.norm(X[:, np.newaxis] - Y, axis=2)
        K = np.exp(-self.intensity * dists)

        if eval_gradient:
            grad_K = (-dists * K)[:, :, np.newaxis]  # Gradient par rapport à intensity
            return K, grad_K

        return K

    def diag(self, X):
        return np.ones(X.shape[0])

    def is_stationary(self):
        return True

    @property
    def hyperparameter_intensity(self):
        return Hyperparameter("intensity", "numeric", (1e-2, 10), fixed=False)


In [ ]:
from scipy.fftpack import fft, fftfreq, rfft, rfftfreq

In [ ]:
YEAR = 365.25
MONTHS6 = YEAR/2.
MONTHS4 = YEAR/3.
QUARTER = YEAR/4. 
DAY = 1.
MONTH = YEAR/12.
WEEK = 7*DAY

In [ ]:
FIGXSIZE_1 = 14
FIGYSIZE_1 = 8

FIGXSIZE_0 = 14
FIGYSIZE_0 = 5

In [ ]:
def plot_gpr_samples(gpr_model, n_samples, ax , x, label):
    """Plot samples drawn from the Gaussian process model.

    If the Gaussian process model is not trained then the drawn samples are
    drawn from the prior distribution. Otherwise, the samples are drawn from
    the posterior distribution. Be aware that a sample here corresponds to a
    function.

    Parameters
    ----------
    gpr_model : `GaussianProcessRegressor`
        A :class:`~sklearn.gaussian_process.GaussianProcessRegressor` model.
    n_samples : int
        The number of samples to draw from the Gaussian process distribution.
    ax : matplotlib axis
        The matplotlib axis where to plot the samples.
    """
    #x = np.linspace(0, 5, 100)
    X = x.reshape(-1, 1)

    y_mean, y_std = gpr_model.predict(X, return_std=True)
    y_samples = gpr_model.sample_y(X, n_samples)

    for idx, single_prior in enumerate(y_samples.T):
        if idx==0:
            ax.plot(
                x,
                single_prior,
                linestyle="--",
                alpha=0.7,
                label=label
            )
        else:
            ax.plot(
                x,
                single_prior,
                linestyle="--",
                alpha=0.7
            )
            
        
        
    ax.plot(x, y_mean, color="black", label="Mean")
    ax.fill_between(
        x,
        y_mean - y_std,
        y_mean + y_std,
        alpha=0.1,
        color="black",
        label=r"$\pm$ 1 std. dev.",
    )
    
    #ax.set_ylim([-3, 3])

In [ ]:
def fourier_analysis(dates, values, ax, mode = "logxlogy",title="Analyse de Fourier - Spectre des fréquences",
                    xlabel="frequency (day)$^{-1}$",ylabel="y-unit $\cdot \sqrt{day}$",label="DFT amplitude", legendout = True, datecut = 59500):
    # Centrer les données autour de la moyenne


    if datecut>0:
        index_selected = np.where(dates >= datecut)[0]
        dates = dates[index_selected]
        values = values[index_selected]
    
    values_centered = values - np.mean(values)

    # Nombre de points
    N = len(dates)
    # Intervalle d'échantillonnage (assume 1 jour entre chaque point)
    T = np.mean(np.diff(dates))  # Période d'échantillonnage

    # signal duration
    Delta_T = dates.max()-dates.min()

    #frequency spacing
    Delta_f = 1/Delta_T
    

    # Fréquence de Nyquist (limite de Shannon)
    f_nyquist = 1 / (2 * T)
    
    # Transformée de Fourier
    fft_values = fft(values_centered)/ np.sqrt(N)
    freqs = fftfreq(N, T)  # Fréquences associées

    # sigma_x
    sigma_x = np.sqrt(np.sum(values_centered**2)/N)

    

    # Seulement la moitié du spectre est utile (symétrie)
    positive_freqs = freqs[:N // 2]
    positive_fft_values = np.abs(fft_values[:N // 2])

    # Calcul de l'écart-type fréquentiel en Hz
    power_spectrum = np.abs(positive_fft_values) ** 2
    sigma_f = np.sqrt(np.sum(positive_freqs**2 * power_spectrum) / np.sum(power_spectrum))
    
    sigma_pp = np.sqrt(np.sum(power_spectrum))
    sigma_x = np.sqrt(np.sum(values_centered**2)/N)

    # Tracer le spectre
    #plt.figure(figsize=(16, 6),layout="constrained")
    
    ax.plot(positive_freqs, positive_fft_values,'ob-' ,ms=5,label=label)

    if mode == "logxliny":
        ax.set_xscale("log")  # Définit l'axe X en échelle logarithmique
        ax.set_yscale("linear")  # Garde l'axe Y en échelle linéaire
    elif mode == "logxlogy":
        ax.set_xscale("log")  # Définit l'axe X en échelle logarithmique
        ax.set_yscale("log")  # Garde l'axe Y en échelle logarithmique
    elif mode == "linxlogy":
        ax.set_xscale("linear")  # Définit l'axe X en  échelle linéaire
        ax.set_yscale("log")  # Garde l'axe Y en échelle logarithmique
    elif mode == "linxliny":
        ax.set_xscale("linear")  # Définit l'axe X en échelle linéaire
        ax.set_yscale("linear")  # Garde l'axe Y en échelle linéaire
        
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    

    ax.axvline(1/YEAR, color='r', linestyle='-', label="Cycle : 365 days - 1 year")
    ax.axvline(1/MONTHS6, color='r', linestyle='--', label="Cycle : 182.6 days - 6 months")
    ax.axvline(1/MONTHS4, color='r', linestyle=':', label="Cycle : 121.7 days - 4 months")
    ax.axvline(1/QUARTER, color='r', linestyle='-.', label="Cycle : 91.3 days - 3 months")
    ax.axvline(1/MONTH, color='r', linestyle=':', label="Cycle : 30.4 days - 1 month")
    ax.axvline(1/WEEK, color='purple', linestyle='--', label="Cycle : 7 days - 1 week")
    ax.axvline(DAY, color='purple', linestyle='-', label="Cycle : 1 day ")
    ax.axvline(1./(0.5*DAY), color='purple', linestyle='-.', label="Cycle : 0.5 day ")

    ax.axvline(f_nyquist, color='g', linestyle='--', label=f"Nyquist frequency({f_nyquist:.3f} cycles/days)")

    if legendout:
        ax.legend(bbox_to_anchor=(1.05, 1.05),fontsize=12)
    else:
        ax.legend(fontsize=10,fancybox=True, framealpha=0.5)

    txtstr_sigma = "$\sigma_x$ = " + f" {sigma_x:0.3f}" 
    txtstr_Ts = "$T_s$ = "  + f" {T:0.3f} days" 
    txtstr_Fs = "$f_s$ = "  + f" {Delta_f:0.3f} / days" 

    txtstr_info = "\n".join([txtstr_sigma,txtstr_Ts ,txtstr_Fs])
    ax.text(0.01, 0.95, txtstr_info, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)
    
    
# Appelle la fonction avec tes données
# fourier_analysis(dates, values)
    return positive_freqs, positive_fft_values, sigma_x,sigma_pp

In [ ]:
def LombScargle_analysis(dates, values, ax ,mode = "logxlogy",title="LombScargle - Spectrum",
                    xlabel="frequency (day)$^{-1}$",ylabel="y-unit",label="Lomb Scargle", legendout = True, datecut = 0 ):
    # Centrer les données autour de la moyenne


    if datecut>0:
        index_selected = np.where(dates >= datecut)[0]
        dates = dates[index_selected]
        values = values[index_selected]

    
    values_centered = values - np.mean(values)
    

    # Nombre de points
    N = len(dates)

    # sigma
    sigma_x = np.sqrt(np.sum(values_centered**2)/N)
    
    # Intervalle d'échantillonnage (assume 1 jour entre chaque point)
    T = np.mean(np.diff(dates))  # Période d'échantillonnage

    # Fréquence de Nyquist (limite de Shannon)
    f_nyquist = 1 / (2 * T)
    

    freqs, power = LombScargle(dates, values_centered).autopower()
   
    ax.plot(freqs, power,'ob-' ,ms=5,label=label)

    if mode == "logxliny":
        ax.set_xscale("log")  # Définit l'axe X en échelle logarithmique
        ax.set_yscale("linear")  # Garde l'axe Y en échelle linéaire
    elif mode == "logxlogy":
        ax.set_xscale("log")  # Définit l'axe X en échelle logarithmique
        ax.set_yscale("log")  # Garde l'axe Y en échelle logarithmique
    elif mode == "linxlogy":
        ax.set_xscale("linear")  # Définit l'axe X en  échelle linéaire
        ax.set_yscale("log")  # Garde l'axe Y en échelle logarithmique
    elif mode == "linxliny":
        ax.set_xscale("linear")  # Définit l'axe X en échelle linéaire
        ax.set_yscale("linear")  # Garde l'axe Y en échelle linéaire
        
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    

    ax.axvline(1/YEAR, color='r', linestyle='-', label="Cycle : 365 days - 1 year")
    ax.axvline(1/MONTHS6, color='r', linestyle='--', label="Cycle : 182.6 days - 6 months")
    ax.axvline(1/MONTHS4, color='r', linestyle=':', label="Cycle : 121.7 days - 4 months")
    ax.axvline(1/QUARTER, color='r', linestyle='-.', label="Cycle : 91.3 days - 3 months")
    ax.axvline(1/MONTH, color='r', linestyle=':', label="Cycle : 30.4 days - 1 month")
    ax.axvline(1/WEEK, color='purple', linestyle='--', label="Cycle : 7 days - 1 week")
    ax.axvline(DAY, color='purple', linestyle='-', label="Cycle : 1 day ")
    ax.axvline(1./(0.5*DAY), color='purple', linestyle='-.', label="Cycle : 0.5 day ")

    ax.axvline(f_nyquist, color='g', linestyle='--', label=f"Nyquist frequency({f_nyquist:.3f} cycles/days)")

    txtstr_sigma = "$\sigma_x$ = " + f" {sigma_x:0.3f}" 
    ax.text(0.01, 0.95, txtstr_sigma, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)
    

    if legendout:
        ax.legend(bbox_to_anchor=(1.05, 1.05),fontsize=12)
    else:
        ax.legend(fontsize=10,fancybox=True, framealpha=0.5)
        


## Configuration

### Residuals

In [ ]:
files_residuals = ["pwv_fitgpresiduals_merra2.csv", 
                   "ozone_fitgpresiduals_merra2.csv", 
                   "vaod_fitgpresiduals_merra2.csv",
                   "angstrom_fitgpresiduals_merra2.csv"] 

# Start analysis

## Analysis of PWV

In [ ]:
index_fn = 0
full_filename = os.path.join(pathdata,files_residuals[index_fn])

In [ ]:
df = pd.read_csv(full_filename,index_col=0)
N = len(df)

In [ ]:
x_full = df["mjd"].values
y_full = df["res"].values
X_full = x_full.reshape(-1, 1)

In [ ]:
tmin_select = 59500
tmax_select = x_full.max()
good_indexes_forresiduals = np.where(np.logical_and(x_full > tmin_select, x_full< tmax_select ))[0]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,3))
ax.hist(y_full[good_indexes_forresiduals],bins=200,facecolor="tab:blue");
ax.set_xlabel("mm")
ax.set_title("input residuals PWV")

In [ ]:
fig = plt.figure(figsize=(FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
#gs = GridSpec(2, 1,figure=fig)
gs = GridSpec(1, 1,figure=fig)
ax1 = fig.add_subplot(gs[0])
#ax2 = fig.add_subplot(gs[1])
        
leg1=ax1.get_legend()
#leg2=ax2.get_legend()
ax1.plot(x_full,y_full,c="b",lw=0.5,label="Merra2")
ax1.set_xlabel("time (MJD)")
ax1.legend()
ax1.set_ylabel("PWV residuals (mm)")
ax1.set_title("PWV residuals after GP-Periodic removal")
figname =f"{pathfigs}/pwv_resGPper_timeseqall_merra2"+figtype
ax1.axvline(tmin_select,color="k",ls=':')
ax1.axvline(tmax_select,color="k",ls=':')
fig.savefig(figname)
plt.show()


In [ ]:
fig,ax = plt.subplots(1,1,figsize=(FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
positive_freqs, positive_fft_values, sigma_x, sigma_pp = fourier_analysis(x_full,y_full,ax=ax ,mode= "logxliny",
                 title = "PWV residuals after GP-Periodic removal (absolute  FFT)",
                 xlabel="frequency (days$^{-1}$)",
                 ylabel=" mm",
                 label="FFT residuals")

txtstr_sigma = "$\sigma_x$ = " + f" {sigma_x:0.2f} mm" 
ax.text(0.01, 0.95, txtstr_sigma, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)

figname =f"{pathfigs}/pwv_resGPper_FFTdata_merra2"+figtype
fig.savefig(figname)
plt.show()

### Define the kernels for PWV

In [ ]:
# Tendance long terme
long_term_trend_kernel = ConstantKernel(.5, (0.0, 10.0)) * RBF(length_scale=365.0)

periodic_1year_kernel =  ConstantKernel(3.0, (0.1, 10.0)) * ExpSineSquared(length_scale= 10*YEAR, periodicity= YEAR,
                                                                           length_scale_bounds="fixed",periodicity_bounds="fixed")
periodic_6months_kernel = ConstantKernel(2.5, (0.1, 10.0)) * ExpSineSquared(length_scale= 20*MONTHS6,periodicity=MONTHS6,
                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed") 
periodic_3months_kernel = ConstantKernel(2.5, (0.1, 10.0)) * ExpSineSquared(length_scale= 40*QUARTER, periodicity=QUARTER,
                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed")
periodic_4months_kernel = ConstantKernel(2.5, (0.1, 10.0)) * ExpSineSquared(length_scale= 30*MONTHS4, periodicity=MONTHS4,
                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed")

seasonal_enveloppe = RBF(length_scale=YEAR, length_scale_bounds=(0.5*YEAR, 10*YEAR))

# Saisonnalité multi-échelle
seasonal_kernel = (
    #seasonal_enveloppe * ( periodic_1year_kernel + periodic_6months_kernel + periodic_3months_kernel)
    #seasonal_enveloppe * ( periodic_1year_kernel  + periodic_3months_kernel)
    #periodic_1year_kernel  + periodic_6months_kernel + periodic_3months_kernel
    periodic_1year_kernel  + periodic_6months_kernel + periodic_4months_kernel + periodic_3months_kernel
    #periodic_1year_kernel
)
# variation journaliere
periodic_1day_kernel = ConstantKernel(2.5, (0., 10.0)) * ExpSineSquared(length_scale= 4*YEAR, periodicity=DAY,
                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed")

#periodic_1day_kernel = ConstantKernel(2.5, (0., 10.0)) *  RBF(length_scale=DAY,length_scale_bounds="fixed")


# Petites fluctuations irrégulières
#irregularities_kernel = ConstantKernel(1.0, (0.0, 10.0)) * RationalQuadratic(length_scale=DAY, alpha=1.0)
#irregularities_kernel = ConstantKernel(1.0, (0.0, 10.0)) * Matern(length_scale=DAY, nu=3.5)
#irregularities_kernel = ConstantKernel(1.0, (0.0, 10.0)) * Matern(length_scale=DAY, nu=1.5) + ConstantKernel(1.0, (0.0, 10.0)) * RationalQuadratic(length_scale=DAY, alpha=1.0)
irregularities_kernel = ConstantKernel(1.0, (0.0, 10.0)) * Matern(length_scale=DAY, nu=0.5) + ConstantKernel(1.0, (0.0, 10.0)) * RationalQuadratic(length_scale=DAY, alpha=1.0)

# with Gamma kernel that is not converging
#irregularities_kernel = ConstantKernel(1.0, (0.0, 10.0)) * RationalQuadratic(length_scale=DAY, alpha=1.0) + ConstantKernel(1.0, (0.0, 10.0)) * GammaKernel(alpha=2.0, length_scale=1.0)

# with Poisson Kernel (slow)
#poisson_kernel = PoissonKernel(intensity=1.0)
#irregularities_kernel = ConstantKernel(1.0, (0.0, 10.0)) * RationalQuadratic(length_scale=DAY, alpha=1.0) + poisson_kernel
#irregularities_kernel = ConstantKernel(1.0, (0.0, 10.0)) * Matern(length_scale=DAY, nu=0.5) + poisson_kernel

# Bruit et variations locales
noise_kernel = ConstantKernel(1.0, (0., 10.0)) * RBF(length_scale=1.0) + WhiteKernel(noise_level=1.0**2,noise_level_bounds="fixed")
#noise_kernel = ConstantKernel(1.0, (0., 10.0)) * RBF(length_scale=1.0) 

# Kernel total
#full_kernel = long_term_trend_kernel + seasonal_kernel + irregularities_kernel + noise_kernel
#full_kernel = seasonal_kernel + irregularities_kernel
#full_kernel = irregularities_kernel + WhiteKernel(noise_level=1.0**2)
#full_kernel = irregularities_kernel + periodic_1day_kernel + noise_kernel
full_kernel = irregularities_kernel * seasonal_enveloppe 

In [ ]:
pwv_kernel = full_kernel

### Make a subsample

In [ ]:
NSAMP = 4000
a = np.arange(0,N ,1)
b = np.random.choice(a, size=NSAMP,replace = False)
index_selected = np.sort(b)

In [ ]:
unique, counts = np.unique(b, return_counts = True)

### subset of values choosen randomly to be fitted on 

In [ ]:
# subset of values choosen randomly to be fitted on 
x = x_full[index_selected]
X = x.reshape(-1, 1)
y = y_full[index_selected]

### Fit GP

In [ ]:
gaussian_process = GaussianProcessRegressor(kernel=pwv_kernel, normalize_y= True)
gaussian_process.fit(X, y)

In [ ]:
txtstr_kernel = f"{gaussian_process.kernel_}"
txtstr_kernel = "\n + ".join(txtstr_kernel.split("+ "))

In [ ]:
txtstr_kernel

### Prediction on subsample

In [ ]:
mjd_min = x_full.min()
mjd_max = x_full.max() + YEAR
mjd_zoom_start = Time("2024-01-01").mjd
mjd_zoom_stop = Time("2025-06-30").mjd

In [ ]:
x_test = np.arange(start=mjd_min, stop=mjd_max,step=2)
X_test = x_test.reshape(-1,1)
mean_y_pred, std_y_pred = gaussian_process.predict(X_test, return_std=True)

In [ ]:
fig = plt.figure(figsize=(FIGXSIZE_1,FIGYSIZE_1),layout="constrained")
gs = GridSpec(2, 1,figure=fig)
#gs = GridSpec(1, 1,figure=fig)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
ax1.plot(x,y,'-',color="k",linestyle="dashed", label="M2 Measurements")
ax1.plot(x_test,mean_y_pred,color="tab:blue", lw=3 ,alpha=1.0, label="Gaussian process")
ax1.fill_between(
    X_test.ravel(),
    mean_y_pred - std_y_pred,
    mean_y_pred + std_y_pred,
    color="tab:blue",
    alpha=0.2,
)
ax1.legend()

ax1.set_ylabel("PWV (mm)")
ax1.set_xlabel("mjd")
ax1.set_title("Fit PWV with Gaussian process (subsample)")
ax1.text(0.1, 0.95, txtstr_kernel, transform=ax1.transAxes, fontsize=16,verticalalignment='top', bbox=props)

ax2.plot(x,y,'-',color="k",linestyle="dashed", label="M2 Measurements")
ax2.plot(x_test,mean_y_pred,color="tab:blue",lw=3 ,alpha=1.0, label="Gaussian process")
ax2.fill_between(
    X_test.ravel(),
    mean_y_pred - std_y_pred,
    mean_y_pred + std_y_pred,
    color="tab:blue",
    alpha=0.2,
)
ax2.legend()

ax2.set_ylabel("PWV (mm)")
ax2.set_xlabel("mjd")
ax2.set_title("Time-Zoom on Fit PWV with Gaussian process")
ax2.set_xlim(mjd_zoom_start,mjd_zoom_stop)
ax2.text(0.1, 0.95, txtstr_kernel, transform=ax2.transAxes, fontsize=16,verticalalignment='top', bbox=props)

figname =f"{pathfigs}/pwv_fitgpPeriodicresidualsubsample_merra2"+figtype
fig.savefig(figname)
plt.show()


In [ ]:
gaussian_process.kernel_

### Residuals on the whole statistics

In [ ]:
mean_yfull_pred, std_yfull_pred = gaussian_process.predict(X_full, return_std=True)

In [ ]:
residuals = y_full -  mean_yfull_pred
residuals_normalized = residuals/std_yfull_pred

In [ ]:
stat_mean = np.mean(residuals[good_indexes_forresiduals])
stat_med = np.median(residuals[good_indexes_forresiduals])
stat_std = np.std(residuals[good_indexes_forresiduals])

In [ ]:
txtstr_stat = [f"mean = {stat_mean:.2f} mm ", f"median = {stat_med:.2f} mm",f"std = {stat_std:.2f} mm"]
txtstr_stat = "\n".join(txtstr_stat)

In [ ]:
fig = plt.figure(figsize=(FIGXSIZE_1,FIGYSIZE_1),layout="constrained")
gs = GridSpec(2, 1,figure=fig)
#gs = GridSpec(1, 1,figure=fig)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
ax1.plot(x_full,y_full,'-',color="k",linestyle="dashed", label="M2 Measurements")
ax1.plot(x_full,mean_yfull_pred,color="tab:blue", lw=3 ,alpha=1.0, label="Gaussian process")
ax1.fill_between(
    X_full.ravel(),
    mean_yfull_pred - std_yfull_pred,
    mean_yfull_pred + std_yfull_pred,
    color="tab:blue",
    alpha=0.2,
)
ax1.legend()

ax1.set_ylabel("PWV (mm)")
ax1.set_xlabel("mjd")
ax1.set_title("Fit PWV with Gaussian process")
ax1.text(0.1, 0.95, txtstr_kernel, transform=ax1.transAxes, fontsize=16,verticalalignment='top', bbox=props)


ax2.plot(x_full,residuals,'-',color="k",linestyle="solid", label="Residuals")
ax2.fill_between(
    X_full.ravel(),
    - std_yfull_pred,
    std_yfull_pred,
    color="tab:blue",
    alpha=0.2,
)
ax2.legend()

ax2.set_ylabel("PWV residuals (mm)")
ax2.set_xlabel("mjd")
ax2.set_title("Residuals on Fit PWV with Gaussian process")
ax2.axhline(0,color="tab:blue",linewidth=3)
ax2.text(0.01, 0.95, txtstr_stat, transform=ax2.transAxes, fontsize=16,verticalalignment='top', bbox=props)


figname =f"{pathfigs}/pwv_fitgpfinalresidualsall_merra2"+figtype
fig.savefig(figname)
plt.show()


#### Jump statistical laws

https://stackoverflow.com/questions/35544233/fit-a-curve-to-a-histogram-in-python

In [ ]:
# Ajustement d'une distribution alpha-stable aux résidus
# use subsample because this fit is too slow
#residuals_subsamples = residuals[index_selected]
#residuals_subsamples_normalized = residuals[index_selected]/std_y_pred
#alpha, beta, loc, scale = levy_stable.fit(residuals_subsamples)
#print(f"Levy Stable process : alpha = {alpha:.3f} , beta = {beta:.3f}, loc = {loc:.3f} , scale = {scale:.3f}")

#### Must clean the residuals from bad dates and bad values

In [ ]:
#good_indexes_forresiduals = np.where(np.logical_and(x_full > 60000, x_full<x_full.max() ))[0]

#### with scipy.stats

In [ ]:
loc_c,scale_c = cauchy.fit(residuals[good_indexes_forresiduals])
loc_l,scale_l = laplace.fit(residuals[good_indexes_forresiduals])
loc_n,scale_n = norm.fit(residuals[good_indexes_forresiduals])

loc_c0,scale_c0 = cauchy.fit(residuals_normalized[good_indexes_forresiduals])
loc_l0,scale_l0 = laplace.fit(residuals_normalized[good_indexes_forresiduals])
loc_n0,scale_n0 = norm.fit(residuals_normalized[good_indexes_forresiduals])

In [ ]:
print("cauchy",loc_c,scale_c)
print("laplace",loc_l,scale_l)
print("norm",loc_n,scale_n) 

print("cauchy0",loc_c0,scale_c0)
print("laplace0",loc_l0,scale_l0)
print("norm0",loc_n0,scale_n0) 

In [ ]:
def plot_histdata_andfit(data,ax, models, fitter,plotdata=True,datacolor="",function_name="gauss"):
    """
    """

    fcolor = {"gauss":"r","lorentz":"g"}
    
    bin_heights, bin_borders = np.histogram(data, bins='auto')
    bin_widths = np.diff(bin_borders)
    bin_centers = bin_borders[:-1] + bin_widths / 2

    # define the statistics to fit
    
    if function_name == "lorentz":
        t_init = models.Lorentz1D() 
    elif function_name == "gauss":
        t_init = models.Gaussian1D()
    else:
        function_name = "gauss"
        t_init = models.Gaussian1D()
        

    # define the fit method
    fit_t = fitter.LevMarLSQFitter()

    #does the fit on histogram data
    t = fit_t(t_init, bin_centers, bin_heights)

    x_interval_for_fit = np.linspace(bin_borders[0], bin_borders[-1], 1000)

    if plotdata:
        ax.bar(bin_centers, bin_heights, width=bin_widths, label='data',color=datacolor)
        
    ax.plot(x_interval_for_fit, t(x_interval_for_fit), label=function_name, c=fcolor[function_name])
    
    ax.set_xlim(bin_borders.min(),bin_borders.max())
    ax.set_ylim(0., bin_heights.max()*1.2)
    ax.legend()

In [ ]:
XXMAX = np.max(np.abs(residuals))
XXMIN = - XXMAX
xx = np.linspace(XXMIN,XXMAX,100)
#rv = levy_stable(alpha, beta)
rv_c = cauchy({'loc':loc_c,'scale':scale_c})
rv_l = laplace({'loc':loc_l,'scale':scale_l})
rv_n = norm({'loc':loc_n,'scale':scale_n})
rv_c0 = cauchy({'loc':loc_c0,'scale':scale_c0})
rv_l0 = laplace({'loc':loc_l0,'scale':scale_l0})
rv_n0 = norm({'loc':loc_n0,'scale':scale_n0})

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,4),layout="constrained")

plot_histdata_andfit(residuals[good_indexes_forresiduals],ax1,models, fitting, plotdata=True,datacolor="tab:blue",function_name="gauss")
plot_histdata_andfit(residuals[good_indexes_forresiduals],ax1,models, fitting,plotdata=False,datacolor="tab:blue",function_name="lorentz")
#ax1.hist(residuals,bins=50,facecolor="tab:blue",density=True)
ax1.set_title("PWV final Residuals")
ax1.text(0.55, 0.95, txtstr_stat, transform=ax1.transAxes, fontsize=12,verticalalignment='top', bbox=props)
ax1.set_xlabel("$\Delta PWV$ (mm)")
#ax1.set_yscale("log")
#ax1.set_xlim(XXMIN,XXMAX)
#ax1.plot(xx, rv.pdf(xx), 'k-', lw=2, label='Levy stable')
#ax1.plot(xx, rv_c.pdf(xx), 'r-', lw=2, label='cauchy')
#ax1.plot(xx, rv_l.pdf(xx), 'g-', lw=2, label='laplace')
#ax1.plot(xx, rv_n.pdf(xx), 'b-', lw=2, label='norm')
ax1.legend(loc="upper left")
ax1.set_ylim(0,2e3)


#ax2.set_ylim(1e-3,1)
ax2.set_title("PWV final Residuals normalized")
#ax2.hist(residuals_normalized,bins=50,facecolor="tab:blue",density=True)
plot_histdata_andfit(residuals_normalized[good_indexes_forresiduals],ax2,models, fitting,plotdata=True,datacolor="tab:blue",function_name="gauss")
plot_histdata_andfit(residuals_normalized[good_indexes_forresiduals],ax2,models, fitting,plotdata=False,datacolor="tab:blue",function_name="lorentz")
#ax2.plot(xx, rv.pdf(xx), 'k-', lw=2, label='Levy stable pdf')
#ax2.plot(xx, rv_c0.pdf(xx), 'r-', lw=2, label='cauchy')
#ax2.plot(xx, rv_l0.pdf(xx), 'g-', lw=2, label='laplace')
#ax2.plot(xx, rv_n0.pdf(xx), 'b-', lw=2, label='norm')
ax2.set_xlabel("$\\frac{\Delta PWV}{\sigma}$")
ax2.legend()
ax2.set_xlim(-10.,10.)
#ax2.set_yscale("log")
ax2.set_ylim(1e0,2e3)

figname =f"{pathfigs}/pwv_finalres_histdata_merra2"+figtype
fig.savefig(figname)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
positive_freqs, positive_fft_values, sigma_x, _ = fourier_analysis(x_full,residuals,ax=ax ,mode= "logxliny",
                 title = "PWV final residuals (absolute  FFT)",
                 xlabel="frequency (days$^{-1}$)",
                 ylabel=" mm",
                 label="FFT residuals")

#txtstr_sigma = "$\sigma$ = " + f" {sigma_x:0.2f} mm"
#ax.text(0.01, 0.95, txtstr_sigma, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)

figname =f"{pathfigs}/pwv_finalres_FFTdata_merra2"+figtype
fig.savefig(figname)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
LombScargle_analysis(x_full,residuals, ax=ax ,mode= "logxliny",
                 title = "LombScargle : PWV residuals",
                 xlabel="frequency (days$^{-1}$)",
                 ylabel=" ",
                 label="Merra2 PWV")
figname =f"{pathfigs}/pwv_finalres_LombScargle_merra2"+figtype
fig.savefig(figname)
plt.show()

## Analysis of Ozone

In [ ]:
index_fn = 1
full_filename = os.path.join(pathdata,files_residuals[index_fn])

In [ ]:
df = pd.read_csv(full_filename,index_col=0)
N = len(df)

In [ ]:
x_full = df["mjd"].values
y_full = df["res"].values
X_full = x_full.reshape(-1, 1)

In [ ]:
tmin_select = 59500
tmax_select = x_full.max()
good_indexes_forresiduals = np.where(np.logical_and(x_full > tmin_select, x_full< tmax_select ))[0]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,3))
ax.hist(y_full[good_indexes_forresiduals],bins=200,facecolor="tab:red");
ax.set_xlabel("DU")
ax.set_title("input residuals Ozone")

In [ ]:
fig = plt.figure(figsize=(FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
#gs = GridSpec(2, 1,figure=fig)
gs = GridSpec(1, 1,figure=fig)
ax1 = fig.add_subplot(gs[0])
#ax2 = fig.add_subplot(gs[1])
        
leg1=ax1.get_legend()
#leg2=ax2.get_legend()
ax1.plot(x_full,y_full,c="tab:red",lw=0.5,label="Merra2")
ax1.set_xlabel("time (MJD)")
ax1.legend()
ax1.set_ylabel("Ozone residuals(DU)")
ax1.set_title("Ozone residuals after GP-Periodic removal")
figname =f"{pathfigs}/ozone_resGPper_timeseqall_merra2"+figtype
fig.savefig(figname)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
positive_freqs, positive_fft_values, sigma_x, _ = fourier_analysis(x_full,y_full,ax=ax ,mode= "logxliny",
                 title = "Ozone residuals after GP-Periodic removal (absolute  FFT)",
                 xlabel="frequency (days$^{-1}$)",
                 ylabel="DU",
                 label="FFT residuals")
figname =f"{pathfigs}/ozone_resGPper_FFTdata_merra2"+figtype

txtstr_sigma = "$\sigma$ = " + f" {sigma_x:0.2f} DU"
ax.text(0.01, 0.95, txtstr_sigma, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)

fig.savefig(figname)
plt.show()

### Define the kernels

In [ ]:
# Tendance long terme
long_term_trend_kernel = ConstantKernel(.5, (0.0, 50.0)) * RBF(length_scale=365.0)

periodic_1year_kernel =  ConstantKernel(3.0, (0.1, 50.0)) * ExpSineSquared(length_scale= 10*YEAR, periodicity= YEAR,
                                                                           length_scale_bounds="fixed",periodicity_bounds="fixed")
periodic_6months_kernel = ConstantKernel(2.5, (0.1, 50.0)) * ExpSineSquared(length_scale= 20*MONTHS6,periodicity=MONTHS6,
                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed") 
periodic_3months_kernel = ConstantKernel(2.5, (0.1, 50.0)) * ExpSineSquared(length_scale= 40*QUARTER, periodicity=QUARTER,
                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed")
periodic_4months_kernel = ConstantKernel(2.5, (0.1, 50.0)) * ExpSineSquared(length_scale= 30*MONTHS4, periodicity=MONTHS4,
                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed")

seasonal_enveloppe = RBF(length_scale=YEAR, length_scale_bounds=(0.5*YEAR, 10*YEAR))

# Saisonnalité multi-échelle
seasonal_kernel = (
    #seasonal_enveloppe * ( periodic_1year_kernel + periodic_6months_kernel + periodic_3months_kernel)
    #seasonal_enveloppe * ( periodic_1year_kernel  + periodic_3months_kernel)
    #periodic_1year_kernel  + periodic_6months_kernel + periodic_3months_kernel
    periodic_1year_kernel  + periodic_6months_kernel + periodic_4months_kernel + periodic_3months_kernel
    #periodic_1year_kernel
)
# variation journaliere
#periodic_1day_kernel = ConstantKernel(2.5, (0., 10.0)) * ExpSineSquared(length_scale= 4*YEAR, periodicity=DAY,
#                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed")

periodic_1day_kernel = ConstantKernel(2.5, (0., 50.0)) *  RBF(length_scale=DAY,length_scale_bounds="fixed")


# Petites fluctuations irrégulières
#irregularities_kernel = ConstantKernel(10.0, (0.0, 50.0)) * RationalQuadratic(length_scale=DAY, alpha=1.0)
#irregularities_kernel = ConstantKernel(1.0, (0.0, 50.0)) * Matern(length_scale=DAY, nu=1.5) +  ConstantKernel(1.0, (0.0, 50.0)) * RationalQuadratic(length_scale=DAY, alpha=1.0)
irregularities_kernel = ConstantKernel(10.0, (0.0, 50.0)) * Matern(length_scale=DAY, nu=0.5) + ConstantKernel(10.0, (0.0, 50.0)) * RationalQuadratic(length_scale=DAY, alpha=1.0)

# with Gamma kernel that is not converging
#irregularities_kernel = ConstantKernel(1.0, (0.0, 10.0)) * RationalQuadratic(length_scale=DAY, alpha=1.0) + ConstantKernel(1.0, (0.0, 10.0)) * GammaKernel(alpha=2.0, length_scale=1.0)

# with Poisson Kernel (slow)
#poisson_kernel = PoissonKernel(intensity=1.0)
#irregularities_kernel = ConstantKernel(1.0, (0.0, 10.0)) * RationalQuadratic(length_scale=DAY, alpha=1.0) + poisson_kernel
#irregularities_kernel = ConstantKernel(1.0, (0.0, 10.0)) * Matern(length_scale=DAY, nu=0.5) + poisson_kernel


# Bruit et variations locales
noise_kernel = ConstantKernel(1.0, (0., 50.0)) * RBF(length_scale=1.0) + WhiteKernel(noise_level=1.0**2)
#noise_kernel = ConstantKernel(1.0, (0., 10.0)) * RBF(length_scale=1.0) 

# Kernel total
#full_kernel = long_term_trend_kernel + seasonal_kernel + irregularities_kernel + noise_kernel
#full_kernel = seasonal_kernel + irregularities_kernel
#full_kernel = irregularities_kernel + WhiteKernel(noise_level=1**2)
#full_kernel = irregularities_kernel + periodic_1day_kernel + noise_kernel
full_kernel = irregularities_kernel  * seasonal_enveloppe

In [ ]:
ozone_kernel = full_kernel

### Make subsample

In [ ]:
NSAMP = 4000
a = np.arange(0,N ,1)
b = np.random.choice(a, size=NSAMP,replace = False)
index_selected = np.sort(b)

In [ ]:
# subset of values choosen randomly to be fitted on 
x = x_full[index_selected]
X = x.reshape(-1, 1)
y = y_full[index_selected]

### Gaussian Fit

In [ ]:
gaussian_process = GaussianProcessRegressor(kernel=ozone_kernel, normalize_y=True)
gaussian_process.fit(X, y)

In [ ]:
txtstr_kernel = f"{gaussian_process.kernel_}"
txtstr_kernel = "\n + ".join(txtstr_kernel.split("+ "))

### Prediction on subsample

In [ ]:
mjd_min = x_full.min()
mjd_max = x_full.max() + YEAR
mjd_zoom_start = Time("2024-01-01").mjd
mjd_zoom_stop = Time("2025-06-30").mjd

In [ ]:
x_test = np.arange(start=mjd_min, stop=mjd_max,step=2)
X_test = x_test.reshape(-1,1)
mean_y_pred, std_y_pred = gaussian_process.predict(X_test, return_std=True)

In [ ]:
fig = plt.figure(figsize=(FIGXSIZE_1,FIGYSIZE_1),layout="constrained")
gs = GridSpec(2, 1,figure=fig)
#gs = GridSpec(1, 1,figure=fig)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
ax1.plot(x,y,'-',color="k",linestyle="dashed", label="M2 Measurements")
ax1.plot(x_test,mean_y_pred,color="tab:red", lw=3,alpha=1.0, label="Gaussian process")
ax1.fill_between(
    X_test.ravel(),
    mean_y_pred - std_y_pred,
    mean_y_pred + std_y_pred,
    color="tab:red",
    alpha=0.2,
)
ax1.legend()
ax1.set_ylabel("Ozone (DU)")
ax1.set_xlabel("mjd")
ax1.set_title("Fit Ozone with Gaussian process (subsamples)")
ax1.text(0.1, 0.95, txtstr_kernel, transform=ax1.transAxes, fontsize=16,verticalalignment='top', bbox=props)


ax2.plot(x,y,'-',color="k",linestyle="dashed", label="M2 Measurements")
ax2.plot(x_test,mean_y_pred,color="tab:red", lw=3,alpha=1.0, label="Gaussian process")
ax2.fill_between(
    X_test.ravel(),
    mean_y_pred - std_y_pred,
    mean_y_pred + std_y_pred,
    color="tab:red",
    alpha=0.2,
)
ax2.legend()

ax2.set_ylabel("Ozone (DU)")
ax2.set_xlabel("mjd")
ax2.set_title("Time-Zoom on Fit Ozone with Gaussian process")
ax2.set_xlim(mjd_zoom_start,mjd_zoom_stop)
ax2.text(0.1, 0.95, txtstr_kernel, transform=ax2.transAxes, fontsize=16,verticalalignment='top', bbox=props)


figname =f"{pathfigs}/ozone_fitgpPeriodicresidualsubsample_merra2"+figtype
fig.savefig(figname)
plt.show()


### Residuals on the whole statistics

In [ ]:
mean_yfull_pred, std_yfull_pred = gaussian_process.predict(X_full, return_std=True)

In [ ]:
residuals = y_full -  mean_yfull_pred
residuals_normalized = residuals/std_yfull_pred

In [ ]:
stat_mean = np.mean(residuals[good_indexes_forresiduals])
stat_med = np.median(residuals[good_indexes_forresiduals])
stat_std = np.std(residuals[good_indexes_forresiduals])

In [ ]:
txtstr_stat = [f"mean = {stat_mean:.2f} DU ", f"median = {stat_med:.2f} DU",f"std = {stat_std:.2f} DU"]
txtstr_stat = "\n".join(txtstr_stat)

In [ ]:
fig = plt.figure(figsize=(FIGXSIZE_1,FIGYSIZE_1),layout="constrained")
gs = GridSpec(2, 1,figure=fig)
#gs = GridSpec(1, 1,figure=fig)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
ax1.plot(x_full,y_full,'-',color="k",linestyle="dashed", label="M2 Measurements")
ax1.plot(x_full,mean_yfull_pred,color="tab:red", lw=3 ,alpha=1.0, label="Gaussian process")
ax1.fill_between(
    X_full.ravel(),
    mean_yfull_pred - std_yfull_pred,
    mean_yfull_pred + std_yfull_pred,
    color="tab:red",
    alpha=0.2,
)
ax1.legend()

ax1.set_ylabel("Ozone (DU)")
ax1.set_xlabel("mjd")
ax1.set_title("Fit Ozone with Gaussian process")
ax1.text(0.1, 0.95, txtstr_kernel, transform=ax1.transAxes, fontsize=16,verticalalignment='top', bbox=props)


ax2.plot(x_full,residuals,'-',color="k",linestyle="solid", label="Residuals")
ax2.fill_between(
    X_full.ravel(),
    - std_yfull_pred,
    std_yfull_pred,
    color="tab:red",
    alpha=0.2,
)
ax2.legend()

ax2.set_ylabel("Ozone residuals (DU)")
ax2.set_xlabel("mjd")
ax2.set_title("Residuals on Fit Ozone with Gaussian process")
ax2.axhline(0,color="tab:red",linewidth=3)
ax2.text(0.01, 0.95, txtstr_stat, transform=ax2.transAxes, fontsize=16,verticalalignment='top', bbox=props)


figname =f"{pathfigs}/ozone_fitgpfinalresidualsall_merra2"+figtype
fig.savefig(figname)
plt.show()


### Statistical Law of jumps

In [ ]:
# Ajustement d'une distribution alpha-stable aux résidus
residuals_subsamples = residuals[index_selected]
#alpha, beta, loc, scale = levy_stable.fit(residuals_subsamples)
#print(f"Levy Stable process : alpha = {alpha:.3f} , beta = {beta:.3f}, loc = {loc:.3f} , scale = {scale:.3f}")

In [ ]:
loc_c,scale_c = cauchy.fit(residuals_subsamples)
loc_l,scale_l = laplace.fit(residuals_subsamples)
loc_n,scale_n = norm.fit(residuals_subsamples)

In [ ]:
print("cauchy",loc_c,scale_c)
print("laplace",loc_l,scale_l)
print("norm",loc_n,scale_n) 

In [ ]:
XXMAX = np.max(np.abs(residuals))
XXMIN = - XXMAX
xx = np.linspace(1.5*XXMIN,1.5*XXMAX,100)
#rv = levy_stable(alpha, beta)
#rv_c = cauchy({'loc':loc_c,'scale':scale_c})
#rv_l = laplace({'loc':loc_l,'scale':scale_l})
#rv_n = norm({'loc':loc_n,'scale':scale_n})
rv_c = cauchy()
rv_l = laplace()
rv_n = norm()

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,4),layout="constrained")

plot_histdata_andfit(residuals[good_indexes_forresiduals],ax1,models, fitting, plotdata=True,datacolor="tab:red",function_name="gauss")
plot_histdata_andfit(residuals[good_indexes_forresiduals],ax1,models, fitting,plotdata=False,datacolor="tab:red",function_name="lorentz")
#ax1.hist(residuals,bins=50,facecolor="tab:blue",density=True)
ax1.set_title("Ozone final Residuals")
ax1.text(0.55, 0.95, txtstr_stat, transform=ax1.transAxes, fontsize=12,verticalalignment='top', bbox=props)
ax1.set_xlabel("$\Delta Ozone$ (DU)")
#ax1.set_yscale("log")
#ax1.set_xlim(XXMIN,XXMAX)
#ax1.plot(xx, rv.pdf(xx), 'k-', lw=2, label='Levy stable')
#ax1.plot(xx, rv_c.pdf(xx), 'r-', lw=2, label='cauchy')
#ax1.plot(xx, rv_l.pdf(xx), 'g-', lw=2, label='laplace')
#ax1.plot(xx, rv_n.pdf(xx), 'b-', lw=2, label='norm')
ax1.legend(loc="upper left")
ax1.set_ylim(0,2e3)


#ax2.set_ylim(1e-3,1)
ax2.set_title("Ozone final Residuals normalized")
#ax2.hist(residuals_normalized,bins=50,facecolor="tab:blue",density=True)
plot_histdata_andfit(residuals_normalized[good_indexes_forresiduals],ax2,models, fitting,plotdata=True,datacolor="tab:red",function_name="gauss")
plot_histdata_andfit(residuals_normalized[good_indexes_forresiduals],ax2,models, fitting,plotdata=False,datacolor="tab:red",function_name="lorentz")
#ax2.plot(xx, rv.pdf(xx), 'k-', lw=2, label='Levy stable pdf')
#ax2.plot(xx, rv_c0.pdf(xx), 'r-', lw=2, label='cauchy')
#ax2.plot(xx, rv_l0.pdf(xx), 'g-', lw=2, label='laplace')
#ax2.plot(xx, rv_n0.pdf(xx), 'b-', lw=2, label='norm')
ax2.set_xlabel("$\\frac{\Delta Ozone}{\sigma}$")
ax2.legend()
ax2.set_xlim(-10.,10.)
#ax2.set_yscale("log")
ax2.set_ylim(1e0,2e3)

figname =f"{pathfigs}/ozone_finalres_histdata_merra2"+figtype
fig.savefig(figname)
plt.show()

In [ ]:
#fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,4),layout="constrained")
#ax1.hist(residuals,bins=50,facecolor="tab:red",density=True)
#ax1.set_title("Ozone final Residuals")
#ax1.text(0.55, 0.95, txtstr_stat, transform=ax1.transAxes, fontsize=12,verticalalignment='top', bbox=props)
#ax1.set_xlabel("$\Delta Ozone$ (DU)")
#ax1.set_yscale("log")
#ax1.set_xlim(XXMIN,XXMAX)
#ax1.plot(xx, rv.pdf(xx), 'k-', lw=2, label='Levy stable')
#ax1.plot(xx, rv_c.pdf(xx), 'r-', lw=2, label='cauchy')
#ax1.plot(xx, rv_l.pdf(xx), 'g-', lw=2, label='laplace')
#ax1.plot(xx, rv_n.pdf(xx), 'b-', lw=2, label='norm')
#ax1.legend(loc="upper left")
#ax1.set_ylim(1e-3,1)

#ax2.plot(xx, rv.pdf(xx), 'k-', lw=2, label='Levy stable pdf')
#ax2.plot(xx, rv_c.pdf(xx), 'r-', lw=2, label='cauchy')
#ax2.plot(xx, rv_l.pdf(xx), 'g-', lw=2, label='laplace')
#ax2.plot(xx, rv_n.pdf(xx), 'b-', lw=2, label='norm')
#ax2.set_ylim(1e-3,1)
#ax2.legend()
#ax2.set_yscale("log")

#figname =f"{pathfigs}/ozone_finalres_histdata_merra2"+figtype
#fig.savefig(figname)
#plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
positive_freqs, positive_fft_values, sigma_x, _ = fourier_analysis(x_full,residuals,ax=ax ,mode= "logxliny",
                 title = "Ozone final residuals (absolute  FFT)",
                 xlabel="frequency (days$^{-1}$)",
                 ylabel="DU $\cdot \sqrt{day}$",
                 label="FFT residuals")

#txtstr_sigma = "$\sigma$ = " + f" {sigma_x:0.2f} DU"
#ax.text(0.01, 0.95, txtstr_sigma, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)

figname =f"{pathfigs}/ozone_finalres_FFTdata_merra2"+figtype
fig.savefig(figname)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
LombScargle_analysis(x_full,residuals, ax=ax ,mode= "logxliny",
                 title = "LombScargle : Ozone residuals",
                 #xlabel="frequency (days$^{-1}$)",
                 ylabel="DU",
                 label="Merra2 Ozone")
figname =f"{pathfigs}/ozone_finalres_LombScargle_merra2"+figtype
fig.savefig(figname)
plt.show()

## Aerosol VAOD

In [ ]:
index_fn = 2
full_filename = os.path.join(pathdata,files_residuals[index_fn])

In [ ]:
df = pd.read_csv(full_filename,index_col=0)
N = len(df)

In [ ]:
x_full = df["mjd"].values
y_full = df["res"].values
X_full = x_full.reshape(-1, 1)

In [ ]:
tmin_select = 59500
tmax_select = x_full.max()
good_indexes_forresiduals = np.where(np.logical_and(x_full > tmin_select, x_full< tmax_select ))[0]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,3))
ax.hist(y_full[good_indexes_forresiduals],bins=200,facecolor="tab:green");
ax.set_xlabel("VAOD")
ax.set_title("input residuals VAOD")

In [ ]:
fig = plt.figure(figsize=(FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
#gs = GridSpec(2, 1,figure=fig)
gs = GridSpec(1, 1,figure=fig)
ax1 = fig.add_subplot(gs[0])
#ax2 = fig.add_subplot(gs[1])
        
leg1=ax1.get_legend()
#leg2=ax2.get_legend()
ax1.plot(x_full,y_full,c="tab:green",lw=0.5,label="Merra2")
ax1.set_xlabel("time (MJD)")
ax1.legend()
ax1.set_ylabel("VAOD residuals")
ax1.set_title("VAOD residuals after GP-Periodic removal (absolute  FFT)")
figname =f"{pathfigs}/vaod_resGPper_timeseqall_merra2"+figtype
fig.savefig(figname)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
positive_freqs, positive_fft_values, sigma_x, _ = fourier_analysis(x_full,y_full,ax=ax ,mode= "logxliny",
                 title = "VAOD residuals after GP-Periodic removal (absolute  FFT)",
                 xlabel="frequency (days$^{-1}$)",
                 ylabel="",
                 label="FFT residuals")

#txtstr_sigma = "$\sigma$ = " + f" {sigma_x:0.3f}"
#ax.text(0.01, 0.95, txtstr_sigma, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)

figname =f"{pathfigs}/vaod_resGPper_FFTdata_merra2"+figtype
fig.savefig(figname)
plt.show()

### Define the kernels

In [ ]:
# Tendance long terme
long_term_trend_kernel = ConstantKernel(.5, (0.0, 50.0)) * RBF(length_scale=365.0)


periodic_1year_kernel =  ConstantKernel(0.1, (0, 2.)) * ExpSineSquared(length_scale= 10*YEAR, periodicity= YEAR,
                                                                           length_scale_bounds="fixed",periodicity_bounds="fixed")
periodic_6months_kernel = ConstantKernel(0.1, (0, 2.)) * ExpSineSquared(length_scale= 20*MONTHS6,periodicity=MONTHS6,
                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed") 
periodic_3months_kernel = ConstantKernel(0.1, (0, 2.)) * ExpSineSquared(length_scale= 40*QUARTER, periodicity=QUARTER,
                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed")
periodic_4months_kernel = ConstantKernel(0.1, (0, 2.)) * ExpSineSquared(length_scale= 30*MONTHS4, periodicity=MONTHS4,
                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed")

periodic_1months_kernel = ConstantKernel(0.1, (0, 2.)) * ExpSineSquared(length_scale= 4+12*MONTH, periodicity=MONTH,
                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed")


seasonal_enveloppe = RBF(length_scale=YEAR, length_scale_bounds=(0.5*YEAR, 10*YEAR))

# Saisonnalité multi-échelle
seasonal_kernel = (
    #seasonal_enveloppe * ( periodic_1year_kernel + periodic_6months_kernel + periodic_3months_kernel)
    #seasonal_enveloppe * ( periodic_1year_kernel  + periodic_3months_kernel)
    #periodic_1year_kernel  + periodic_6months_kernel + periodic_3months_kernel
    periodic_1year_kernel  + periodic_6months_kernel + periodic_4months_kernel + periodic_3months_kernel
    #periodic_1year_kernel
)
# variation journaliere
#periodic_1day_kernel = ConstantKernel(2.5, (0., 10.0)) * ExpSineSquared(length_scale= 4*YEAR, periodicity=DAY,
#                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed")

periodic_1day_kernel = ConstantKernel(2.5, (0., 50.0)) *  RBF(length_scale=DAY,length_scale_bounds="fixed")


# Petites fluctuations irrégulières
#irregularities_kernel = ConstantKernel(.1, (0.0, 5.0)) * RationalQuadratic(length_scale=DAY, alpha=1.0)
#irregularities_kernel = ConstantKernel(.1, (0.0, 5.0)) * Matern(length_scale=DAY, nu=1.5) +  ConstantKernel(.1, (0.0, 5.0)) * RationalQuadratic(length_scale=DAY, alpha=1.0)
irregularities_kernel = ConstantKernel(0.1, (0.0, 5.0)) * Matern(length_scale=DAY, nu=0.5) + ConstantKernel(.1, (0.0, 5.0)) * RationalQuadratic(length_scale=DAY, alpha=1.0)



# Bruit et variations locales
noise_kernel = ConstantKernel(0.1, (0., 5.0)) * RBF(length_scale=1.0) + WhiteKernel(noise_level=1.0**2)
#noise_kernel = ConstantKernel(1.0, (0., 10.0)) * RBF(length_scale=1.0) 

# Kernel total
#full_kernel = long_term_trend_kernel + seasonal_kernel + irregularities_kernel + noise_kernel
#full_kernel = seasonal_kernel + irregularities_kernel
#full_kernel = irregularities_kernel + WhiteKernel(noise_level=1**2)
#full_kernel = irregularities_kernel + periodic_1day_kernel + noise_kernel
full_kernel = irregularities_kernel * seasonal_enveloppe

In [ ]:
vaod_kernel = (
    full_kernel
)

### Make a subsample

In [ ]:
NSAMP = 4000
a = np.arange(0,N ,1)
b = np.random.choice(a, size=NSAMP,replace = False)
index_selected = np.sort(b)

In [ ]:
# subset of values choosen randomly to be fitted on 
x = x_full[index_selected]
X = x.reshape(-1, 1)
y = y_full[index_selected]

### Fit the gaussian process

In [ ]:
gaussian_process = GaussianProcessRegressor(kernel=vaod_kernel, normalize_y=True)
gaussian_process.fit(X, y)

In [ ]:
txtstr_kernel = f"{gaussian_process.kernel_}"
txtstr_kernel = "\n + ".join(txtstr_kernel.split("+ "))

### Prediction

In [ ]:
mjd_min = x_full.min()
mjd_max = x_full.max() + YEAR
mjd_zoom_start = Time("2024-01-01").mjd
mjd_zoom_stop = Time("2025-06-30").mjd

In [ ]:
x_test = np.arange(start=mjd_min, stop=mjd_max,step=2)
X_test = x_test.reshape(-1,1)
mean_y_pred, std_y_pred = gaussian_process.predict(X_test, return_std=True)

In [ ]:
fig = plt.figure(figsize=(FIGXSIZE_1,FIGYSIZE_1),layout="constrained")
gs = GridSpec(2, 1,figure=fig)
#gs = GridSpec(1, 1,figure=fig)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
ax1.plot(x,y,'-',color="k",linestyle="dashed", label="M2 Measurements")
ax1.plot(x_test,mean_y_pred,color="tab:green", lw=3,alpha=1.0, label="Gaussian process")
ax1.fill_between(
    X_test.ravel(),
    mean_y_pred - std_y_pred,
    mean_y_pred + std_y_pred,
    color="tab:green",
    alpha=0.2,
)
ax1.legend()
ax1.text(0.1, 0.95, txtstr_kernel, transform=ax1.transAxes, fontsize=12,verticalalignment='top', bbox=props)

ax1.set_ylabel("VAOD")
ax1.set_xlabel("mjd")
ax1.set_title("Fit Aerosol VAOD with Gaussian process")

ax2.plot(x,y,'-',color="k",linestyle="dashed", label="M2 Measurements")
ax2.plot(x_test,mean_y_pred,color="tab:green", lw=3 ,alpha=1.0, label="Gaussian process")
ax2.fill_between(
    X_test.ravel(),
    mean_y_pred - std_y_pred,
    mean_y_pred + std_y_pred,
    color="tab:green",
    alpha=0.2,
)
ax2.legend()

ax2.set_ylabel("VAOD")
ax2.set_xlabel("mjd")
ax2.set_title("Time-Zoom on Fit VAOD with Gaussian process")
ax2.set_xlim(mjd_zoom_start,mjd_zoom_stop)
ax2.text(0.1, 0.95, txtstr_kernel, transform=ax2.transAxes, fontsize=12,verticalalignment='top', bbox=props)


figname =f"{pathfigs}/vaod_fitgpPeriodicresidualsubsample_merra2"+figtype
fig.savefig(figname)
plt.show()


### Residuals on the whole statistics

In [ ]:
mean_yfull_pred, std_yfull_pred = gaussian_process.predict(X_full, return_std=True)

In [ ]:
residuals = y_full -  mean_yfull_pred
residuals_normalized = residuals/std_yfull_pred

In [ ]:
stat_mean = np.mean(residuals[good_indexes_forresiduals])
stat_med = np.median(residuals[good_indexes_forresiduals])
stat_std = np.std(residuals[good_indexes_forresiduals])

In [ ]:
txtstr_stat = [f"mean = {stat_mean:.3f}", f"median = {stat_med:.3f}",f"std = {stat_std:.3f}"]
txtstr_stat = "\n".join(txtstr_stat)

In [ ]:
fig = plt.figure(figsize=(FIGXSIZE_1,FIGYSIZE_1),layout="constrained")
gs = GridSpec(2, 1,figure=fig)
#gs = GridSpec(1, 1,figure=fig)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
ax1.plot(x_full,y_full,'-',color="k",linestyle="dashed", label="M2 Measurements")
ax1.plot(x_full,mean_yfull_pred,color="tab:green", lw=3 ,alpha=1.0, label="Gaussian process")
ax1.fill_between(
    X_full.ravel(),
    mean_yfull_pred - std_yfull_pred,
    mean_yfull_pred + std_yfull_pred,
    color="tab:green",
    alpha=0.2,
)
ax1.legend()

ax1.set_ylabel("VAOD residuals")
ax1.set_xlabel("mjd")
ax1.set_title("Fit VAOD residuals with Gaussian process")
ax1.text(0.1, 0.95, txtstr_kernel, transform=ax1.transAxes, fontsize=12,verticalalignment='top', bbox=props)


ax2.plot(x_full,residuals,'-',color="k",linestyle="solid", label="Residuals")
ax2.fill_between(
    X_full.ravel(),
    - std_yfull_pred,
    std_yfull_pred,
    color="tab:green",
    alpha=0.2,
)
ax2.legend()

ax2.set_ylabel("VAOD residuals")
ax2.set_xlabel("mjd")
ax2.set_title("Residuals on Fit VAOD with Gaussian process")
ax2.axhline(0,color="tab:green",linewidth=3)
ax2.text(0.01, 0.95, txtstr_stat, transform=ax2.transAxes, fontsize=12,verticalalignment='top', bbox=props)

figname =f"{pathfigs}/vaod_fitgpfinalresidualsall_merra2"+figtype

fig.savefig(figname)
plt.show()


### Statistical Law of jumps

In [ ]:
# Ajustement d'une distribution alpha-stable aux résidus
#residuals_subsamples = residuals[index_selected]
#alpha, beta, loc, scale = levy_stable.fit(residuals_subsamples)
#print(f"Levy Stable process : alpha = {alpha:.3f} , beta = {beta:.3f}, loc = {loc:.3f} , scale = {scale:.3f}")

In [ ]:
params_c = cauchy.fit(residuals_subsamples)
params_l = laplace.fit(residuals_subsamples)
params_n = norm.fit(residuals_subsamples)

loc_c,scale_c = params_c
loc_l,scale_l = params_l
loc_n,scale_n = params_n

In [ ]:
print("cauchy",loc_c,scale_c)
print("laplace",loc_l,scale_l)
print("norm",loc_n,scale_n) 

In [ ]:
XXMAX = np.max(np.abs(residuals))
XXMIN = - XXMAX
xx = np.linspace(1.5*XXMIN,1.5*XXMAX,100)
#rv = levy_stable(alpha, beta)
rv_c = cauchy({'loc':loc_c,'scale':scale_c})
rv_l = laplace({'loc':loc_l,'scale':scale_l})
rv_n = norm({'loc':loc_n,'scale':scale_n})
#rv_c = cauchy(params_c)
#rv_l = laplace(params_l)
#rv_n = norm(params_n)

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,4),layout="constrained")

plot_histdata_andfit(residuals[good_indexes_forresiduals],ax1,models, fitting, plotdata=True,datacolor="tab:green",function_name="gauss")
plot_histdata_andfit(residuals[good_indexes_forresiduals],ax1,models, fitting,plotdata=False,datacolor="tab:green",function_name="lorentz")
#ax1.hist(residuals,bins=50,facecolor="tab:blue",density=True)
ax1.set_title("VAOD final Residuals")
ax1.text(0.55, 0.95, txtstr_stat, transform=ax1.transAxes, fontsize=12,verticalalignment='top', bbox=props)
ax1.set_xlabel("$\Delta VAOD$")
#ax1.set_yscale("log")
#ax1.set_xlim(XXMIN,XXMAX)
#ax1.plot(xx, rv.pdf(xx), 'k-', lw=2, label='Levy stable')
#ax1.plot(xx, rv_c.pdf(xx), 'r-', lw=2, label='cauchy')
#ax1.plot(xx, rv_l.pdf(xx), 'g-', lw=2, label='laplace')
#ax1.plot(xx, rv_n.pdf(xx), 'b-', lw=2, label='norm')
ax1.legend(loc="upper left")
ax1.set_ylim(0,1e3)


#ax2.set_ylim(1e-3,1)
ax2.set_title("VAOD final Residuals normalized")
#ax2.hist(residuals_normalized,bins=50,facecolor="tab:blue",density=True)
plot_histdata_andfit(residuals_normalized[good_indexes_forresiduals],ax2,models, fitting,plotdata=True,datacolor="tab:green",function_name="gauss")
plot_histdata_andfit(residuals_normalized[good_indexes_forresiduals],ax2,models, fitting,plotdata=False,datacolor="tab:green",function_name="lorentz")
#ax2.plot(xx, rv.pdf(xx), 'k-', lw=2, label='Levy stable pdf')
#ax2.plot(xx, rv_c0.pdf(xx), 'r-', lw=2, label='cauchy')
#ax2.plot(xx, rv_l0.pdf(xx), 'g-', lw=2, label='laplace')
#ax2.plot(xx, rv_n0.pdf(xx), 'b-', lw=2, label='norm')
ax2.set_xlabel("$\\frac{\Delta VAOD}{\sigma}$")
ax2.legend()
ax2.set_xlim(-10.,10.)
#ax2.set_yscale("log")
ax2.set_ylim(1e0,1e3)

figname =f"{pathfigs}/vaod_finalres_histdata_merra2"+figtype
fig.savefig(figname)
plt.show()

In [ ]:
#fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,4),layout="constrained")
#ax1.hist(residuals,bins=50,facecolor="tab:green",density=True)
#ax1.set_title("VAOD final Residuals")
#ax1.text(0.55, 0.95, txtstr_stat, transform=ax1.transAxes, fontsize=12,verticalalignment='top', bbox=props)
#ax1.set_xlabel("$\Delta VAOD$")
#ax1.set_yscale("log")
#ax1.set_xlim(XXMIN,XXMAX)
#ax1.plot(xx, rv.pdf(xx), 'k-', lw=2, label='Levy stable')
#ax1.plot(xx, rv_c.pdf(xx), 'r-', lw=2, label='cauchy')
#ax1.plot(xx, rv_l.pdf(xx), 'g-', lw=2, label='laplace')
#ax1.plot(xx, rv_n.pdf(xx), 'b-', lw=2, label='norm')
#ax1.legend(loc="upper left")
#ax1.set_ylim(1e-3,100)

#ax2.plot(xx, rv.pdf(xx), 'k-', lw=2, label='Levy stable pdf')
#ax2.plot(xx, rv_c.pdf(xx), 'r-', lw=2, label='cauchy')
#ax2.plot(xx, rv_l.pdf(xx), 'g-', lw=2, label='laplace')
#ax2.plot(xx, rv_n.pdf(xx), 'b-', lw=2, label='norm')
#ax2.set_ylim(1e-3,100)
#ax2.legend()
#ax2.set_yscale("log")

#figname =f"{pathfigs}/vaod_finalres_histdata_merra2"+figtype
#fig.savefig(figname)
#plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
positive_freqs, positive_fft_values, sigma_x, _ = fourier_analysis(x_full,residuals,ax=ax ,mode= "logxliny",
                 title = "VAOD final residuals (absolute  FFT)",
                 #xlabel="frequency (days$^{-1}$)",
                 ylabel="",
                 label="FFT residuals")

#txtstr_sigma = "$\sigma$ = " + f" {sigma_x:0.3f}"
#ax.text(0.01, 0.95, txtstr_sigma, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)

figname =f"{pathfigs}/vaod_finalres_FFTdata_merra2"+figtype
fig.savefig(figname)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
LombScargle_analysis(x_full,residuals, ax=ax ,mode= "logxliny",
                 title = "LombScargle : VAOD residuals",
                 xlabel="frequency (days$^{-1}$)",
                 #ylabel="VAOD",
                 label="Merra2 VAOD")
figname =f"{pathfigs}/vaod_finalres_LombScargle_merra2"+figtype
fig.savefig(figname)
plt.show()

## Aerosol Angstrom

In [ ]:
index_fn = 3
full_filename = os.path.join(pathdata,files_residuals[index_fn])

In [ ]:
df = pd.read_csv(full_filename,index_col=0)
N = len(df)

In [ ]:
x_full = df["mjd"].values
y_full = df["res"].values
X_full = x_full.reshape(-1, 1)

In [ ]:
tmin_select = 59500
tmax_select = x_full.max()
good_indexes_forresiduals = np.where(np.logical_and(x_full > tmin_select, x_full< tmax_select ))[0]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,3))
ax.hist(y_full[good_indexes_forresiduals],bins=200,facecolor="tab:purple");
ax.set_xlabel("angstrom")
ax.set_title("input residuals Angstrom")

In [ ]:
fig = plt.figure(figsize=(FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
#gs = GridSpec(2, 1,figure=fig)
gs = GridSpec(1, 1,figure=fig)
ax1 = fig.add_subplot(gs[0])
#ax2 = fig.add_subplot(gs[1])
        
leg1=ax1.get_legend()
#leg2=ax2.get_legend()
ax1.plot(x_full,y_full,c="tab:purple",lw=1,label="Merra2")
ax1.set_xlabel("time (MJD)")
ax1.legend()
ax1.set_ylabel("Angstrom residuals")
ax1.set_title("Angstrom residuals wrt periodic fit")
figname =f"{pathfigs}/angstrom_resGPper_timeseqall_merra2"+figtype
fig.savefig(figname)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
positive_freqs, positive_fft_values, sigma_x, _ = fourier_analysis(x_full,y_full,ax=ax ,mode= "logxliny",
                 title = "Angstrom residuals after GP-Periodic removal (absolute  FFT)",
                 xlabel="frequency (days$^{-1}$)",
                 ylabel="",
                 label="FFT residuals")

#txtstr_sigma = "$\sigma$ = " + f" {sigma_x:0.3f}"
#ax.text(0.01, 0.95, txtstr_sigma, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)

figname =f"{pathfigs}/angstrom_resGPper_FFTdata_merra2"+figtype
fig.savefig(figname)
plt.show()

### Define kernels

In [ ]:
# Tendance long terme
long_term_trend_kernel = ConstantKernel(1., (0.0, 4.0)) * RBF(length_scale=365.0)


periodic_1year_kernel =  ConstantKernel(1., (0, 4.)) * ExpSineSquared(length_scale= 10*YEAR, periodicity= YEAR,
                                                                           length_scale_bounds="fixed",periodicity_bounds="fixed")
periodic_6months_kernel = ConstantKernel(1., (0, 4.)) * ExpSineSquared(length_scale= 20*MONTHS6,periodicity=MONTHS6,
                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed") 
periodic_3months_kernel = ConstantKernel(1., (0, 4.)) * ExpSineSquared(length_scale= 40*QUARTER, periodicity=QUARTER,
                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed")
periodic_4months_kernel = ConstantKernel(1., (0, 4.)) * ExpSineSquared(length_scale= 30*MONTHS4, periodicity=MONTHS4,
                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed")

periodic_1months_kernel = ConstantKernel(1., (0, 4.)) * ExpSineSquared(length_scale= 4+12*MONTH, periodicity=MONTH,
                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed")


seasonal_enveloppe = RBF(length_scale=YEAR, length_scale_bounds=(0.5*YEAR, 10*YEAR))

# Saisonnalité multi-échelle
seasonal_kernel = (
    #seasonal_enveloppe * ( periodic_1year_kernel + periodic_6months_kernel + periodic_3months_kernel)
    #seasonal_enveloppe * ( periodic_1year_kernel  + periodic_3months_kernel)
    #periodic_1year_kernel  + periodic_6months_kernel + periodic_3months_kernel
    periodic_1year_kernel  + periodic_6months_kernel + periodic_4months_kernel + periodic_3months_kernel
    #periodic_1year_kernel
)
# variation journaliere
#periodic_1day_kernel = ConstantKernel(2.5, (0., 10.0)) * ExpSineSquared(length_scale= 4*YEAR, periodicity=DAY,
#                                                                            length_scale_bounds="fixed",periodicity_bounds="fixed")

periodic_1day_kernel = ConstantKernel(1., (0., 4.0)) *  RBF(length_scale=DAY,length_scale_bounds="fixed")


# Petites fluctuations irrégulières
#irregularities_kernel = ConstantKernel(1., (0.0, 4.0)) * RationalQuadratic(length_scale=DAY, alpha=1.0)
#irregularities_kernel = ConstantKernel(1, (0.0, 4.0)) * Matern(length_scale=DAY, nu=1.5) +  ConstantKernel(1., (0.0, 4.)) * RationalQuadratic(length_scale=DAY, alpha=1.0)
irregularities_kernel = ConstantKernel(1.0, (0.0, 4.0)) * Matern(length_scale=DAY, nu=0.5) + ConstantKernel(1.0, (0.0, 4.0)) * RationalQuadratic(length_scale=DAY, alpha=1.0)



# Bruit et variations locales
noise_kernel = ConstantKernel(1., (0., 4.0)) * RBF(length_scale=1.0) + WhiteKernel(noise_level=1.0**2)
#noise_kernel = ConstantKernel(1.0, (0., 10.0)) * RBF(length_scale=1.0) 

# Kernel total
#full_kernel = long_term_trend_kernel + seasonal_kernel + irregularities_kernel + noise_kernel
#full_kernel = seasonal_kernel + irregularities_kernel
#full_kernel = irregularities_kernel + WhiteKernel(noise_level=1**2)
#full_kernel = irregularities_kernel + periodic_1day_kernel + noise_kernel
full_kernel = irregularities_kernel * seasonal_enveloppe

In [ ]:
angstrom_kernel = (
    full_kernel
)

### subsample

In [ ]:
NSAMP = 4000
a = np.arange(0,N ,1)
b = np.random.choice(a, size=NSAMP,replace = False)
index_selected = np.sort(b)

In [ ]:
# subset of values choosen randomly to be fitted on 
x = x_full[index_selected]
X = x.reshape(-1, 1)
y = y_full[index_selected]

### Fit the gaussian process model

In [ ]:
gaussian_process = GaussianProcessRegressor(kernel=angstrom_kernel, normalize_y= True)
gaussian_process.fit(X, y )

In [ ]:
txtstr_kernel = f"{gaussian_process.kernel_}"
txtstr_kernel = "\n + ".join(txtstr_kernel.split("+ "))

### Prediction

In [ ]:
mjd_min = x_full.min()
mjd_max = x_full.max() + YEAR
mjd_zoom_start = Time("2024-01-01").mjd
mjd_zoom_stop = Time("2025-06-30").mjd

In [ ]:
x_test = np.arange(start=mjd_min, stop=mjd_max,step=2)
X_test = x_test.reshape(-1,1)
mean_y_pred, std_y_pred = gaussian_process.predict(X_test, return_std=True)

In [ ]:
fig = plt.figure(figsize=(FIGXSIZE_1,FIGYSIZE_1),layout="constrained")
gs = GridSpec(2, 1,figure=fig)
#gs = GridSpec(1, 1,figure=fig)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
ax1.plot(x,y,'-',color="k",linestyle="dashed", label="M2 Measurements")
ax1.plot(x_test,mean_y_pred,color="tab:purple",lw=3 ,alpha=1.0, label="Gaussian process")
ax1.fill_between(
    X_test.ravel(),
    mean_y_pred - std_y_pred,
    mean_y_pred + std_y_pred,
    color="tab:purple",
    alpha=0.2,
)
ax1.legend()

ax1.set_ylabel("Angstrom")
ax1.set_xlabel("mjd")
ax1.set_title("Fit Aerosol Angstrom exponent with Gaussian process")
ax1.text(0.1, 0.95, txtstr_kernel, transform=ax1.transAxes, fontsize=12,verticalalignment='top', bbox=props)

ax2.plot(x,y,'-',color="k",linestyle="dashed", label="M2 Measurements")
ax2.plot(x_test,mean_y_pred,color="tab:purple", lw=3, alpha=1.0, label="Gaussian process")
ax2.fill_between(
    X_test.ravel(),
    mean_y_pred - std_y_pred,
    mean_y_pred + std_y_pred,
    color="tab:purple",
    alpha=0.2,
)
ax2.legend()


ax2.set_ylabel("Angstrom")
ax2.set_xlabel("mjd")
ax2.set_title("Time-Zoom on Angstrom exponent with Gaussian process")
ax2.set_xlim(mjd_zoom_start,mjd_zoom_stop)
ax2.text(0.1, 0.95, txtstr_kernel, transform=ax2.transAxes, fontsize=12,verticalalignment='top', bbox=props)


figname =f"{pathfigs}/angstrom_fitgpPeriodicresidualsubsample_merra2"+figtype
fig.savefig(figname)
plt.show()


### Residuals on the whole statistics

In [ ]:
mean_yfull_pred, std_yfull_pred = gaussian_process.predict(X_full, return_std=True)

In [ ]:
residuals = y_full -  mean_yfull_pred
residuals_normalized = residuals/std_yfull_pred

In [ ]:
stat_mean = np.mean(residuals[good_indexes_forresiduals])
stat_med = np.median(residuals[good_indexes_forresiduals])
stat_std = np.std(residuals[good_indexes_forresiduals])

In [ ]:
txtstr_stat = [f"mean = {stat_mean:.3f}", f"median = {stat_med:.3f}",f"std = {stat_std:.3f}"]
txtstr_stat = "\n".join(txtstr_stat)

In [ ]:
fig = plt.figure(figsize=(FIGXSIZE_1,FIGYSIZE_1),layout="constrained")
gs = GridSpec(2, 1,figure=fig)
#gs = GridSpec(1, 1,figure=fig)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1],sharey=ax1)
ax1.plot(x_full,y_full,'-',color="k",linestyle="dashed", label="M2 Measurements")
ax1.plot(x_full,mean_yfull_pred,color="tab:purple", lw=3 ,alpha=1.0, label="Gaussian process")
ax1.fill_between(
    X_full.ravel(),
    mean_yfull_pred - std_yfull_pred,
    mean_yfull_pred + std_yfull_pred,
    color="tab:purple",
    alpha=0.2,
)
ax1.legend()

ax1.set_ylabel("Angstrom exponent")
ax1.set_xlabel("mjd")
ax1.set_title("Fit Angstrom with Gaussian process")
ax1.text(0.3, 0.95, txtstr_kernel, transform=ax1.transAxes, fontsize=12,verticalalignment='top', bbox=props)


ax2.plot(x_full,residuals,'-',color="k",linestyle="solid", label="Residuals")
ax2.fill_between(
    X_full.ravel(),
    - std_yfull_pred,
    std_yfull_pred,
    color="tab:purple",
    alpha=0.2,
)
ax2.legend()

ax2.set_ylabel("Angstrom residuals")
ax2.set_xlabel("mjd")
ax2.set_title("Residuals on Fit Angstrom exponent with Gaussian process")
ax2.axhline(0,color="tab:purple",linewidth=3)
ax2.text(0.01, 0.95, txtstr_stat, transform=ax2.transAxes, fontsize=12,verticalalignment='top', bbox=props)


figname =f"{pathfigs}/angstrom_fitgpfinalresidualsall_merra2"+figtype
fig.savefig(figname)
plt.show()


### Statistical law of jumps

In [ ]:
# Ajustement d'une distribution alpha-stable aux résidus
residuals_subsamples = residuals[index_selected]
#alpha, beta, loc, scale = levy_stable.fit(residuals_subsamples)
#print(f"Levy Stable process : alpha = {alpha:.3f} , beta = {beta:.3f}, loc = {loc:.3f} , scale = {scale:.3f}")

In [ ]:
loc_c,scale_c = cauchy.fit(residuals_subsamples)
loc_l,scale_l = laplace.fit(residuals_subsamples)
loc_n,scale_n = norm.fit(residuals_subsamples)

In [ ]:
print("cauchy",loc_c,scale_c)
print("laplace",loc_l,scale_l)
print("norm",loc_n,scale_n) 

In [ ]:
XXMAX = np.max(np.abs(residuals))
XXMIN = - XXMAX
xx = np.linspace(1.5*XXMIN,1.5*XXMAX,100)
#rv = levy_stable(alpha, beta)
#rv_c = cauchy({'loc':loc_c,'scale':scale_c})
#rv_l = laplace({'loc':loc_l,'scale':scale_l})
#rv_n = norm({'loc':loc_n,'scale':scale_n})
rv_c = cauchy()
rv_l = laplace()
rv_n = norm()

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,4),layout="constrained")

plot_histdata_andfit(residuals[good_indexes_forresiduals],ax1,models, fitting, plotdata=True,datacolor="tab:purple",function_name="gauss")
plot_histdata_andfit(residuals[good_indexes_forresiduals],ax1,models, fitting,plotdata=False,datacolor="tab:purple",function_name="lorentz")
#ax1.hist(residuals,bins=50,facecolor="tab:blue",density=True)
ax1.set_title("Angstrom final Residuals")
ax1.text(0.55, 0.95, txtstr_stat, transform=ax1.transAxes, fontsize=12,verticalalignment='top', bbox=props)
ax1.set_xlabel("$\Delta Angstrom$")
#ax1.set_yscale("log")
#ax1.set_xlim(XXMIN,XXMAX)
#ax1.plot(xx, rv.pdf(xx), 'k-', lw=2, label='Levy stable')
#ax1.plot(xx, rv_c.pdf(xx), 'r-', lw=2, label='cauchy')
#ax1.plot(xx, rv_l.pdf(xx), 'g-', lw=2, label='laplace')
#ax1.plot(xx, rv_n.pdf(xx), 'b-', lw=2, label='norm')
ax1.legend(loc="upper left")
ax1.set_ylim(0,2e3)


#ax2.set_ylim(1e-3,1)
ax2.set_title("Angstrom final Residuals normalized")
#ax2.hist(residuals_normalized,bins=50,facecolor="tab:blue",density=True)
plot_histdata_andfit(residuals_normalized[good_indexes_forresiduals],ax2,models, fitting,plotdata=True,datacolor="tab:purple",function_name="gauss")
plot_histdata_andfit(residuals_normalized[good_indexes_forresiduals],ax2,models, fitting,plotdata=False,datacolor="tab:purple",function_name="lorentz")
#ax2.plot(xx, rv.pdf(xx), 'k-', lw=2, label='Levy stable pdf')
#ax2.plot(xx, rv_c0.pdf(xx), 'r-', lw=2, label='cauchy')
#ax2.plot(xx, rv_l0.pdf(xx), 'g-', lw=2, label='laplace')
#ax2.plot(xx, rv_n0.pdf(xx), 'b-', lw=2, label='norm')
ax2.set_xlabel("$\\frac{\Delta Angstrom}{\sigma}$")
ax2.legend()
ax2.set_xlim(-10.,10.)
#ax2.set_yscale("log")
ax2.set_ylim(1e0,2e3)

figname =f"{pathfigs}/angstrom_finalres_histdata_merra2"+figtype
fig.savefig(figname)
plt.show()

In [ ]:
#fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,4),layout="constrained")
#ax1.hist(residuals,bins=50,facecolor="tab:purple",density=True)
#ax1.set_title("Angstrom final Residuals")
#ax1.text(0.55, 0.95, txtstr_stat, transform=ax1.transAxes, fontsize=12,verticalalignment='top', bbox=props)
#ax1.set_xlabel("$\Delta$ Angstrom")
#ax1.set_yscale("log")
#ax1.set_xlim(XXMIN,XXMAX)
#ax1.plot(xx, rv.pdf(xx), 'k-', lw=2, label='Levy stable')
#ax1.plot(xx, rv_c.pdf(xx), 'r-', lw=2, label='cauchy')
#ax1.plot(xx, rv_l.pdf(xx), 'g-', lw=2, label='laplace')
#ax1.plot(xx, rv_n.pdf(xx), 'b-', lw=2, label='norm')
#ax1.legend(loc="upper left")
#ax1.set_ylim(1e-3,100)

#ax2.plot(xx, rv.pdf(xx), 'k-', lw=2, label='Levy stable pdf')
#ax2.plot(xx, rv_c.pdf(xx), 'r-', lw=2, label='cauchy')
#ax2.plot(xx, rv_l.pdf(xx), 'g-', lw=2, label='laplace')
#ax2.plot(xx, rv_n.pdf(xx), 'b-', lw=2, label='norm')
#ax2.set_ylim(1e-3,100)
#ax2.legend()
#ax2.set_yscale("log")

#figname =f"{pathfigs}/angstrom_finalres_histdata_merra2"+figtype
#fig.savefig(figname)
#plt.show()

In [ ]:
#fig,ax = plt.subplots(1,1,figsize=(6,4),layout="constrained")
#ax.hist(residuals,bins=50,facecolor="tab:purple")
#ax.set_title("Residuals to Angstrol GP periodic model")
#ax.text(0.45, 0.95, txtstr_stat, transform=ax.transAxes, fontsize=12,verticalalignment='top', bbox=props)
#ax.set_xlabel("$\Delta$ Angstrom")
#figname =f"{pathfigs}/angstrom_finalres_histdata_merra2"+figtype
#fig.savefig(figname)
#plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
positive_freqs, positive_fft_values, sigma_x, _ = fourier_analysis(x_full,residuals,ax=ax ,mode= "logxliny",
                 title = "Angstrom final residuals (absolute  FFT)",
                 #xlabel="frequency (days$^{-1}$)",
                 ylabel="",
                 label="FFT residuals")

#txtstr_sigma = "$\sigma$ = " + f" {sigma_x:0.3f}"
#ax.text(0.01, 0.95, txtstr_sigma, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)

figname =f"{pathfigs}/angstrom_finalres_FFTdata_merra2"+figtype
fig.savefig(figname)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
LombScargle_analysis(x_full,residuals, ax=ax ,mode= "logxliny",
                 title = "LombScargle : Angstrom exponent residuals",
                 #xlabel="frequency (days$^{-1}$)",
                 ylabel="Angstrom exponent",
                 label="Merra2 Angstrom exponent")
figname =f"{pathfigs}/angstrom_finalres_LombScargle_merra2"+figtype
fig.savefig(figname)
plt.show()